In [32]:
## imports
import pandas as pd
import tensorflow as tf
from os import *
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [33]:
import numpy as np

In [53]:
csv = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

df = pd.DataFrame()

wanted_cols = ['Offense Start DateTime', 'Offense Parent Group', 'Offense']

for col in wanted_cols:
    df[col] = csv[col]
    df = df.dropna()

df["Month"] = [pd.to_datetime(i).month for i in df['Offense Start DateTime']]

df["Day"] = [pd.to_datetime(i).day for i in df['Offense Start DateTime']]

df["Year"] = [pd.to_datetime(i).year for i in df['Offense Start DateTime']]

df.drop(columns = ['Offense Start DateTime'], inplace = True)

dfs = {}

In [64]:
def compress(frame: pd.DataFrame, wanted_columns: list, category: str) -> pd.DataFrame:
    df = pd.DataFrame()
    df[category] = frame[category]
    for col in wanted_columns:
        df[col] = frame[col]
    return df.value_counts().to_frame()

In [66]:
for i in ['Offense Parent Group', 'Offense']:
    dfs[i] = {}
    dfs[i]["ymd"] = compress(df, ["Year", "Month", "Day"], i)
    dfs[i]["ym"] = compress(df, ["Year", "Month"], i)
    dfs[i]["y"] = compress(df, ["Year"], i)

In [68]:
dfs # MARIA SHOULD LOOK AT THIS

{'Offense Parent Group': {'ymd':                                         0
  Offense Parent Group  Year Month Day     
  FRAUD OFFENSES        2020 5     18   593
                             3     8    521
                             5     15   507
                                   14   457
                                   11   411
  ...                                   ...
  HOMICIDE OFFENSES     2009 8     27     1
                                   23     1
                                   9      1
                             7     26     1
  WEAPON LAW VIOLATIONS 2023 2     13     1
  
  [87153 rows x 1 columns],
  'ym':                                                     0
  Offense Parent Group                 Year Month      
  FRAUD OFFENSES                       2020 5      6519
  LARCENY-THEFT                        2021 10     2753
                                       2022 3      2653
                                            1      2624
                        

In [63]:
compress(df, ["Year", "Month", "Day"], "Offense")

0
Offense                  Year Month Day     
Identity Theft           2020 5     18   585
                              3     8    516
                              5     15   500
                                    14   453
                                    11   406
...                                      ...
Fondling                 2018 1     29     1
Stolen Property Offenses 2022 6     20     1
Fondling                 2018 1     23     1
Stolen Property Offenses 2022 6     22     1
Aggravated Assault       1997 3     12     1

[139112 rows x 1 columns]

In [59]:
df.value_counts().to_frame()

0
Offense Parent Group        Offense                     Month Day Year     
FRAUD OFFENSES              Identity Theft              5     18  2020  585
                                                        3     8   2020  516
                                                        5     15  2020  500
                                                              14  2020  453
                                                              11  2020  406
...                                                                     ...
FAMILY OFFENSES, NONVIOLENT Family Offenses, Nonviolent 7     8   2010    1
                                                              7   2018    1
                                                                  2016    1
                                                                  2015    1
ANIMAL CRUELTY              Animal Cruelty              1     1   2023    1

[139112 rows x 1 columns]

In [47]:
df

,Offense Start DateTime,Offense Parent Group,Offense
0,02/05/2020 10:10:00 AM,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations
1,02/03/2020 08:00:00 AM,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories
2,02/02/2020 08:30:00 PM,ROBBERY,Robbery
3,02/05/2020 01:17:00 AM,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property
4,02/05/2020 12:51:21 AM,DRIVING UNDER THE INFLUENCE,Driving Under the Influence
...,...,...,...
1033410,06/26/2013 11:00:00 AM,MOTOR VEHICLE THEFT,Motor Vehicle Theft
1033411,02/14/2012 03:04:00 PM,LARCENY-THEFT,Shoplifting
1033412,02/20/2011 08:00:00 PM,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering
1033413,09/19/2010 04:59:00 PM,LARCENY-THEFT,Shoplifting


In [42]:
csv.columns

Index(['Report Number', 'Offense ID', 'Offense Start DateTime',
       'Offense End DateTime', 'Report DateTime', 'Group A B',
       'Crime Against Category', 'Offense Parent Group', 'Offense',
       'Offense Code', 'Precinct', 'Sector', 'Beat', 'MCPP',
       '100 Block Address', 'Longitude', 'Latitude'],
      dtype='object')

In [39]:
df['Offense Start DateTime'].isna().value_counts()

False    1032212
Name: Offense Start DateTime, dtype: int64

In [27]:
## datasets, data cleanup, test-train split
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

## cleaning up the dates to have month, day, year columns
## also splitting it up into the layers of data
## which includes year, month, day

ds = ds[ds['Offense Start DateTime'] != None]

# dateTime = ds['Offense Start DateTime'].dropna()

ds["Month"] = [pd.to_datetime(i).month for i in ds['Offense Start DateTime']]

ds["Day"] = [pd.to_datetime(i).day for i in ds['Offense Start DateTime']]

ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

ds.loc[:, 'Crime Against Category':'Year']

,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Month,Day,Year
0,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2.0,5.0,2020.0
1,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2.0,3.0,2020.0
2,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2.0,2.0,2020.0
3,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2.0,5.0,2020.0
4,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2.0,5.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033410,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,W,Q,Q3,QUEEN ANNE,1ST AVE N / VALLEY ST,-122.355386,47.626251,6.0,26.0,2013.0
1033411,PROPERTY,LARCENY-THEFT,Shoplifting,23C,N,U,U1,ROOSEVELT/RAVENNA,73XX BLOCK OF ROOSEVELT WAY NE,-122.317464,47.681894,2.0,14.0,2012.0
1033412,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,S,S3,RAINIER VIEW,100XX BLOCK OF M L KING JR WAY S,-122.278352,47.510453,2.0,20.0,2011.0
1033413,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M3,DOWNTOWN COMMERCIAL,5XX BLOCK OF PIKE ST,-122.335018,47.610909,9.0,19.0,2010.0


In [ ]:
## train-test split
## to-do: for dsYear, Month, Day, create 3 splits
## each of 3 splits will be split 75% train and 25% test

In [ ]:
## polynomial regression
## to-do: learn how to do polynomial regression ,_,

In [ ]:
s = 'jacbja'

print(s[0:s.find('a')])
print(s[0:s.find('b')][0:s.find('a')])

j


In [5]:
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

In [20]:
ds['Offense Start DateTime'].dropna()

0          02/05/2020 10:10:00 AM
1          02/03/2020 08:00:00 AM
2          02/02/2020 08:30:00 PM
3          02/05/2020 01:17:00 AM
4          02/05/2020 12:51:21 AM
                    ...          
1033410    06/26/2013 11:00:00 AM
1033411    02/14/2012 03:04:00 PM
1033412    02/20/2011 08:00:00 PM
1033413    09/19/2010 04:59:00 PM
1033414    02/25/2010 06:00:00 PM
Name: Offense Start DateTime, Length: 1032212, dtype: object

In [17]:
ds['Report DateTime'].isnull().value_counts()

False    1033415
Name: Report DateTime, dtype: int64

In [25]:
ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

In [26]:
ds

,Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Month,Day,Year
0,2020-044620,12605873663,02/05/2020 10:10:00 AM,NaN,02/05/2020 11:24:31 AM,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2.0,5.0,2020.0
1,2020-044452,12605598696,02/03/2020 08:00:00 AM,02/04/2020 08:00:00 AM,02/05/2020 10:06:28 AM,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2.0,3.0,2020.0
2,2020-044465,12605567653,02/02/2020 08:30:00 PM,02/02/2020 09:30:00 PM,02/05/2020 09:39:33 AM,A,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2.0,2.0,2020.0
3,2020-044225,12605174036,02/05/2020 01:17:00 AM,02/05/2020 02:21:00 AM,02/05/2020 03:30:55 AM,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2.0,5.0,2020.0
4,2020-044076,12605081469,02/05/2020 12:51:21 AM,NaN,02/05/2020 12:51:31 AM,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2.0,5.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033410,2013-227022,7682354808,06/26/2013 11:00:00 AM,06/29/2013 06:30:00 AM,06/29/2013 09:43:00 AM,A,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,W,Q,Q3,QUEEN ANNE,1ST AVE N / VALLEY ST,-122.355386,47.626251,6.0,26.0,2013.0
1033411,2012-045494,7672915592,02/14/2012 03:04:00 PM,NaN,02/14/2012 03:04:00 PM,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,N,U,U1,ROOSEVELT/RAVENNA,73XX BLOCK OF ROOSEVELT WAY NE,-122.317464,47.681894,2.0,14.0,2012.0
1033412,2011-061224,41519143023,02/20/2011 08:00:00 PM,02/21/2011 01:55:00 PM,02/21/2011 01:55:00 PM,A,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,S,S3,RAINIER VIEW,100XX BLOCK OF M L KING JR WAY S,-122.278352,47.510453,2.0,20.0,2011.0
1033413,2010-328592,7692227482,09/19/2010 04:59:00 PM,NaN,09/19/2010 04:59:00 PM,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M3,DOWNTOWN COMMERCIAL,5XX BLOCK OF PIKE ST,-122.335018,47.610909,9.0,19.0,2010.0


In [ ]:
def compress_bitch(date_wanted: str = category: str) -> pd.DataFrame:
    